In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
# roi = ee.Feature(ee.FeatureCollection('users/311605001111/YangtzeCity/wuhan').first()).geometry()
# Map.addLayer(roi, {}, "roi")
# Map.centerObject(roi,8)

# # 青海省部分地区
roi = ee.Geometry.Rectangle([89.8565, 34.6503,91.4664, 35.8282])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(img):
    image = img.clip(roi)
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)

def area_after_clip(image):
    image_area = image.select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
    imageareas = image_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })    
    return image.set({'imagearea': imageareas.get('B2')})

# 筛选得出采样影像

In [5]:
landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31') \
    .map(maskL8sr) \
    .map(water_index)
Map.addLayer(landsat_images.mosaic(),visParams,'landsat image')

image_area = landsat_images.mosaic().select('B2').gt(0).multiply(ee.Image.pixelArea()).divide(1e6)
areas = image_area.reduceRegion(**{
    'reducer': ee.Reducer.sum(),
    'geometry': roi,
    'scale': 1000,
    'maxPixels': 1e14,
})

# 筛选影像，影像覆盖研究区的范围 > 研究区的1/10
sample_image = landsat_images.map(area_after_clip).filter(ee.Filter.gt('imagearea',ee.Number(areas.get('B2')).divide(7)))
dates = sample_image.aggregate_array('system:time_start').map(lambda d: ee.Date(d).format('YYYY-MM-dd'))
print(sample_image.size().getInfo())
print(dates.getInfo())

39
['2018-01-11', '2018-02-12', '2018-03-16', '2018-04-01', '2018-04-17', '2018-05-03', '2018-06-04', '2018-07-22', '2018-01-02', '2018-01-18', '2018-02-03', '2018-03-07', '2018-03-23', '2018-04-08', '2018-04-24', '2018-05-10', '2018-05-26', '2018-06-27', '2018-08-30', '2018-11-02', '2018-11-18', '2018-12-04', '2018-01-18', '2018-02-03', '2018-03-07', '2018-03-23', '2018-04-08', '2018-04-24', '2018-05-10', '2018-05-26', '2018-06-27', '2018-07-13', '2018-07-29', '2018-08-30', '2018-10-01', '2018-11-02', '2018-11-18', '2018-12-04', '2018-12-20']


# 样本采集函数的定义

In [6]:
lt2_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(roi) \
    .filterDate('2018-01-01', '2018-12-31')
# print(lt2_images.size().getInfo())

In [7]:
## 导入采集的样本集
points_collection = ee.FeatureCollection('users/311605001111/qinghai_2018')
print(points_collection.first().getInfo())
print('total sample number:{}'.format(points_collection.size().getInfo()))


# RandomForest预测使用的波段
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','mNDWI','ndvi_mndwi','AWEI']
# 分类标签
label = 'waterclass'
trainedClassifier = ee.Classifier.smileRandomForest(20).train(points_collection,label,bands)
# train_accuracy = trainedClassifier.confusionMatrix()
# print(train_accuracy.getInfo())
# print(train_accuracy.accuracy().getInfo())
# print(train_accuracy.kappa().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [90.6397502534006, 35.140012738196056]}, 'id': '000000000000000003c3', 'properties': {'AWEI': -5345, 'B2': 1231, 'B3': 1824, 'B4': 2401, 'B5': 2985, 'B6': 3863, 'B7': 3456, 'NDVI': 0.10842926055, 'mNDWI': -0.3585370183, 'ndvi_mndwi': 0.4669662714, 'waterclass': 0}}
total sample number:2199


In [10]:
a = points_collection.filter(ee.Filter.eq('waterclass',1))
print(a.size().getInfo())

86


## 水体频率（校正）

In [ ]:
#对影像进行分类,计算水体面积
def area_of_training(image):
    classified_image = water_index(maskL8sr(image)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    water_area = classified_image.multiply(ee.Image.pixelArea()).divide(1e6)
    waterarea = water_area.reduceRegion(**{
        'reducer': ee.Reducer.sum(),
        'geometry': image.geometry(),
        'scale': 1000,
        'maxPixels': 1e14,
    })
    return image.set({'waterarea': waterarea.get('waterclass')})  

# 需要校正的图像
corrected_image = lt2_images.map(area_of_training).filter(ee.Filter.gt('waterarea',10))
# 不需要校正的图像
No_corrected_image = lt2_images.map(area_of_training).filter(ee.Filter.lte('waterarea',10))

In [ ]:
def AutomaticCorrection(img):
    image = water_index(maskL8sr(img)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')
    basemap = ee.Image.constant(0).toFloat().clip(img.geometry()).clip(roi).rename('waterclass')
    class_image = ee.ImageCollection([basemap,image.remap([0,1],[1,2]).rename('waterclass')]).sum()
    water = class_image.eq(2).selfMask()
    no_data = class_image.eq(0).selfMask()
    occurrence = ee.Image('JRC/GSW1_3/GlobalSurfaceWater').select('occurrence')
    occurrence_water = occurrence.updateMask(water)
    occurrence_no_data = occurrence.updateMask(no_data)
    occurrence_HistogramCount = occurrence_water.reduceRegion(**{
        'reducer': ee.Reducer.histogram(100,1),
        'geometry': img.clip(roi).geometry(),
        'scale': 30,
        'bestEffort': True
    })
    histogram = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('histogram'))
    bucketMeans = ee.List(ee.Dictionary(occurrence_HistogramCount.get('occurrence')).get('bucketMeans'))    
    count_threshold = ee.Number(histogram.reduce(ee.Reducer.sum())).multiply(0.0017)
    index = histogram.map(lambda i : ee.Algorithms.If(ee.Number(i).gte(ee.Number(count_threshold)),ee.Number(i))).removeAll([None]).get(0)
    occurrence_threshold = bucketMeans.get(histogram.indexOf(index))
    occurrence_corrected_water = occurrence_no_data.gte(ee.Number(occurrence_threshold)).selfMask().select('occurrence').rename('waterclass')
    enhanced_water = ee.ImageCollection([basemap,water,occurrence_corrected_water]).sum()
    return enhanced_water.set({'occurrence_threshold':occurrence_threshold})

### 分母为有效观测次数

In [ ]:
corrected_img = corrected_image.map(AutomaticCorrection).sum()
No_corrected_img = No_corrected_image.map(lambda i : water_index(maskL8sr(i)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')).sum()
# Map.addLayer(corrected_img,{'palette':['white','green'],'min':0,'max':60},"corrected_img")
# Map.addLayer(No_corrected_img,{'palette':['white','green'],'min':0,'max':60},"No_corrected_img")
waterPixel_image = ee.ImageCollection([corrected_img,No_corrected_img]).sum()
# Map.addLayer(waterPixel_image,{'palette':['white','blue'],'min':0,'max':60},"waterPixel_image")

validPixel_image = lt2_images.map(lambda i : maskL8sr(i)).count().select('B3').clip(roi).rename('count')
valid_waterfrequency = waterPixel_image.select('waterclass').divide(validPixel_image.select('count')).rename('frequency')
Map.addLayer(valid_waterfrequency,{'palette':['white','green'],'min':0,'max':1},"valid frequency cor")

valid_permanentwater = valid_waterfrequency.gte(0.75)
Map.addLayer(valid_permanentwater.selfMask(),{'palette':['blue']},"valid permanent water cor")

### 分母为全部观测次数

In [ ]:
corrected_img = corrected_image.map(AutomaticCorrection).sum()
No_corrected_img = No_corrected_image.map(lambda i : water_index(maskL8sr(i)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')).sum()
# Map.addLayer(corrected_img,{'palette':['white','green'],'min':0,'max':60},"corrected_img")
# Map.addLayer(No_corrected_img,{'palette':['white','green'],'min':0,'max':60},"No_corrected_img")
waterPixel_image = ee.ImageCollection([corrected_img,No_corrected_img]).sum()
# Map.addLayer(waterPixel_image,{'palette':['white','blue'],'min':0,'max':60},"waterPixel_image")

validPixel_image = lt2_images.count().select('B3').clip(roi).rename('count')
all_waterfrequency = waterPixel_image.select('waterclass').divide(validPixel_image.select('count')).rename('frequency')
Map.addLayer(all_waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency cor")

all_permanentwater = all_waterfrequency.gte(0.75)
Map.addLayer(all_permanentwater.selfMask(),{'palette':['cyan']},"permanent water cor")

## 水体频率（未校正）

In [ ]:
waterPixel_count = lt2_images.map(lambda i : water_index(maskL8sr(i)).select(bands).classify(trainedClassifier).eq(1).rename('waterclass')).sum()
# Map.addLayer(waterpixel_count,{'palette':['white','#e40775'],'min':0,'max':35},"waterPixelCount")

validPixel_count = lt2_images.map(lambda i : maskL8sr(i)).count().select('B3').clip(roi).rename('count')
waterfrequency = waterPixel_count.select('waterclass').divide(validPixel_count.select('count')).rename('frequency')
Map.addLayer(waterfrequency,{'palette':['white','green'],'min':0,'max':1},"frequency")

permanentwater = waterfrequency.gte(0.75)
Map.addLayer(permanentwater.selfMask(),{'palette':['red']},"permanent_water")

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
%pylab inline
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'min':0, 'max':1}
fig = plt.figure(figsize=(12, 8))
cmap = 'Blues'
ax = cartoee.get_map(valid_waterfrequency, region=region, vis_params=vis,cmap = cmap)
cartoee.add_colorbar(ax, vis,cmap=cmap,loc="right",label="water frequency", orientation="vertical")
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'waterbody frequency(Auto-Correction/validPixel/2018)', fontsize=20)

In [ ]:
# permanent waterbody
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8565, 34.6503,91.4664, 35.8282] 
vis = {'palette':['D3D3D3','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(valid_permanentwater, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'permanent water(Auto-Correction/validPixel/2018)', fontsize=20)

# 附录

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.8565, 34.6503,91.4664, 35.8282]
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(landsat_images.mosaic(), region=region, vis_params=visParams)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'landsat image(2018)', fontsize=20)

In [ ]:
# 制图
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [89.812623, 34.507942, 91.5449, 35.6369]
vis = {'palette':['D4D4D4','blue'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(enhanced_water, region=region, vis_params=vis)
# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.2,0.2], linestyle=":")
ax.set_title(label = 'classified image(AutomaticCorrection)', fontsize=20)

In [ ]:
# index_waterclass
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['cccccc', '99d9ea', '0000ff'],'min':0,'max':2}
fig = plt.figure(figsize=(12, 8))
# use cartoee to get a map
ax = cartoee.get_map(index_waterclass, region=region, vis_params=vis)

# add gridlines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'waterbody frequency(INDEX) in 2018', fontsize=20)

In [ ]:
# landExtent
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
from geemap import cartoee
region = [113.6, 29.9, 115.1, 31.4]
vis = {'palette':['white','cccccc'],'min':0,'max':1}
fig = plt.figure(figsize=(12, 8))
ax = cartoee.get_map(landExtent, region=region, vis_params=vis)
cartoee.add_gridlines(ax, interval=[0.5,0.5], linestyle=":")
ax.set_title(label = 'land extent in 2018', fontsize=20)